## Running structure predictions with multimer and monomeric predictions

Before running this notebook, please ensure you:

1. Are logged in by running `dm login EMAIL` in the terminal
2. Have a Token registered and saved on the file


In [ ]:
!pip install deepmirror
# !dm login <YOUREMAIL>

In [ ]:
import time
import zipfile

import py3Dmol

import deepmirror.api as api

### Define some helper functions

In [ ]:
def cofold(chains: list[dict], constraint_settings: dict) -> str:
    response = api.structure_prediction(chains, constraint_settings)
    task_id = response["task_id"]

    while True:
        response = api.get_structure_prediction(task_id)
        status = response["status"]
        if status == "completed":
            break
        print(f"Current status: {status} - Waiting 2 min for completion...")
        time.sleep(120)

    with open(f"result-{task_id}.zip", "wb") as f:
        f.write(api.download_structure_prediction(task_id))

    return task_id


def view_results(task_id: str):
    with zipfile.ZipFile(f"result-{task_id}.zip", "r") as zf:
        cif_data = zf.read("data.cif").decode("utf-8")
    view = py3Dmol.view(width=400, height=400)
    view.addModel(cif_data, "cif")
    view.setStyle({"cartoon": {}})
    view.addStyle({"hetflag": True}, {"stick": {}})
    view.zoomTo()
    return view

# Protein + Ligand

In [ ]:
chains = [
    {
        "label": "A",
        "value": "MTEYKLVVVGADGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHHYREQIKRVKDSEDVPMVLVGNKCDLPSRTVDTKQAQDLARSYGIPFIETSAKTRQGVDDAFYTLVREIRKHKEK",
        "type": "protein",
    },
    {
        "label": "B",
        "value": "c1nc2c(n1C3C(C(C(O3)COP(=O)(O)OP(=O)(O)O)O)O)N=C(NC2=O)N",
        "type": "ligand",
    },
]


## Define each point - point constraint. This is used to define constraints between any two residues or a protein and a small molecule within a pocket

* Please note that the residue indices are 1-indexed
* Connection Type: "contact", used to define residue contacts within or between protein chains, "pocket" used for small molecule and protein pocket
* min_distance_angstron: minimum distance between the heavy atoms of the interacting residues
* max_distance_angstron: maximum distance between the heavy atoms of the interacting residues
* confidence: Weight assigned to the constraint
* comment: User defined comment for the constraint.
* restraint_id: restraint_i, where i is the number assigned to the constraint, starting from 0
* Please define as many constraints and add them in the final list of constraints

##### Protein Contact Map constraint: connection_type: 'contact'

In [ ]:
constraint_1 = {
    "chainA": "A",
    "res_idxA": "F28",
    "chainB": "A",
    "res_idxB": "Q99",
    "connection_type": "contact",
    "confidence": 1.0,
    "min_distance_angstrom": 3.0,
    "max_distance_angstrom": 5.0,
    "comment": "Inter Protein Chain Contact",
    "restraint_id": "restraint_0",
}


##### Protein - Small Molecule Pocket constraint: connection_type: 'pocket'. 
Please make sure that the small molecule is assigned the chainA and the protein is assigned the chainB for defining pocket restraint


In [ ]:
constraint_2 = {
    "chainA": "B",
    "res_idxA": "",
    "chainB": "A",
    "res_idxB": "D57",
    "connection_type": "pocket",
    "confidence": 1.0,
    "min_distance_angstrom": 4,
    "max_distance_angstrom": 5,
    "comment": "Protein - Small Molecule Pocket Interaction",
    "restraint_id": "restraint_1",
}

##### Combine all constraints into one list to set up constraint_settings

In [ ]:
constraints = [constraint_1, constraint_2]
constraint_settings = {"constraints": constraints}

### Initiate Structure Prediction

In [ ]:
task_id = cofold(chains, constraint_settings)

In [ ]:
view = view_results(task_id)
view